In [1]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit
from qiskit_ibm_provider import IBMProvider

import numpy as np
import pandas as pd
import time

/opt/conda/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/conda/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
provider = IBMProvider()

backend = provider.get_backend('ibmq_qasm_simulator')

In [3]:
class QKD:
    def __init__(self, qubits, backend=backend):
        self.bits = qubits
        self.backend = backend

    def generate_random_bits(self):
        circuit = QuantumCircuit(1, 1)
        circuit.h(0)
        circuit.measure(0, 0)
        job = self.backend.run(circuit, shots=self.bits, memory=True)
        result = job.result()
        memory = result.get_memory(circuit)
        random_bits = [int(m) for m in memory]
        return random_bits
    
    def alice_sending(self):
        alice_bits = self.generate_random_bits()
        alice_bases = self.generate_random_bits()
        alice_circuit = QuantumCircuit(self.bits)
        for i in range(self.bits):
            if alice_bits[i] == 1:
                alice_circuit.x(i)  # Apply X-gate if the bit is 1
            if alice_bases[i] == 1:
                alice_circuit.h(i)  # Apply H-gate if the base is 1
        return alice_bits, alice_bases, alice_circuit
    
    def bob_receiving(self):
        alice_bits, alice_bases, alice_circuit = self.alice_sending()
        bob_bases = self.generate_random_bits()
        for i in range(self.bits):
            if bob_bases[i] == 1:
                alice_circuit.h(i)
        alice_circuit.measure_all()
        return alice_bits, alice_bases, bob_bases, alice_circuit
        
    def generate_key(self):
        alice_bits, alice_bases, bob_bases, final_circuit = self.bob_receiving()
        result = self.backend.run(final_circuit, shots=1).result()
        measurements = result.get_counts()
        bob_bits = [int(bit) for bit in list(measurements.keys())[0][::-1]]

        alice_key = []
        bob_key = []
        for i in range(self.bits):
            if alice_bases[i] == bob_bases[i]:
                alice_key.append(alice_bits[i])
                bob_key.append(bob_bits[i])
        return alice_key, bob_key

In [4]:
# # Assume the backend is properly initialized if needed, like AerSimulator
# num_bits = 15
# qkd_instance = QKD(qubits=num_bits)

# # Generate keys
# alice_key, bob_key = qkd_instance.generate_key()
# print("Alice's Key:", alice_key)
# print("Bob's Key:", bob_key)

In [5]:
num_qubits = list(range(2, 26))
results = []

current_time = time.strftime("%H:%M:%S", time.localtime())
print(f"Starting: {current_time}")

for qubits in num_qubits:
    mean_length = []
    successes = 0
    for _ in range(10):
        qkd_instance = QKD(qubits=qubits)
        alice_key, bob_key = qkd_instance.generate_key()
        mean_length.append(len(alice_key))
        if alice_key == bob_key:
            successes += 1
    
    average_length = np.mean(mean_length)  # Calculate the average key length
    success_rate = successes / 10  # Calculate the success rate

    # Store the results in a list of dictionaries
    results.append({
        'Qubits': qubits,
        'Average Key Length': average_length,
        'Success Rate': success_rate
    })
    
    current_time = time.strftime("%H:%M:%S", time.localtime())
    print(f"{qubits} qubits done at: {current_time}")
    
current_time = time.strftime("%H:%M:%S", time.localtime())
print(f"Finished: {current_time}")

Starting: 05:39:53
2 qubits done at: 05:43:18
3 qubits done at: 05:46:22
4 qubits done at: 05:49:05
5 qubits done at: 05:51:54
6 qubits done at: 05:55:07
7 qubits done at: 05:58:09
8 qubits done at: 06:01:34
9 qubits done at: 06:05:22
10 qubits done at: 06:08:37
11 qubits done at: 06:11:37
12 qubits done at: 06:14:45
13 qubits done at: 06:17:55
14 qubits done at: 06:21:17
15 qubits done at: 06:24:25
16 qubits done at: 06:27:22
17 qubits done at: 06:30:26
18 qubits done at: 06:33:29
19 qubits done at: 06:36:27
20 qubits done at: 06:40:24
21 qubits done at: 06:43:40
22 qubits done at: 06:47:24
23 qubits done at: 06:51:08
24 qubits done at: 06:54:32
25 qubits done at: 06:58:04
Finished: 06:58:04


In [6]:
df = pd.DataFrame(results)

# Save the DataFrame to a txt file
df.to_csv('qkd_results_simulator.txt', index=False)